In [1]:
# Add erpsc to path
import os
import sys

In [2]:
import nltk
import itertools 

In [3]:
my_path = '/Users/vassiki/Desktop/nhw17/DataDrivenCognitiveOntology/'
#sys.path.append(os.getcwd)
sys.path.append(my_path)
# ^ Update the above with a link to the folder that has 'erpsc' in it, if not currently in path

In [4]:
# Import erpsc
from lisc.base import Base
from lisc.core.db import SCDB
from lisc.data import Data
from lisc.data_all import DataAll

In [5]:
# Add data to path
db = SCDB(False)
#fake_path = '/Users/vassiki/Desktop/nhw17/ERPSC_NLP/notebooks'
db.project_path = os.path.join(os.getcwd(), 'dat')
#db.project_path = fake_path
db.gen_paths()

In [6]:
# Load raw data from a particular ERP
dat = Data('p300')
dat.load(db)

In [7]:
def get_corpus(ds):
    """
    To flatten all abstract words into a long list 
    
    TO DO: Generalize to making a corpus of authors, or any metric the author wants
    
    Attributes
    ----------
    ds : Data structure returned by ERFSC when a query is submitted
    """
    aPreCorpus = []
    
    numArticles = ds.n_articles
    print('Iterating over %d articles for extracting words') %(ds.n_articles)
    for AIdx in xrange(numArticles):
        if ds.words[AIdx] != None:
            aPreCorpus.append(ds.words[AIdx])
        
    
    PreText = list(itertools.chain.from_iterable(aPreCorpus))
    
    Corpus = nltk.Text(PreText)
    
    return Corpus
        

In [8]:
def get_common_words(Corpus,NumWords):
    """
    Return the most common words in the concatenated article text corpus
    
    Attributes
    ----------
    Corpus: generated from keyword arguments from cognitive atlas
    NumWords: Top N words to return
    """
    
    assert(NumWords <= len(Corpus))
    
    fdist = nltk.FreqDist(Corpus)
    common_words = fdist.most_common(NumWords)
    header = [('Term', 'Frequency'),('-'*len('Term'),'-'*len('Frequency'))]
    wordList = header + common_words
    
    width = max(len(e) for t in wordList for e in t[:-1]) + 1 
    format=('%%-%ds' % width) * len(wordList[0])
    print '\n'.join(format % tuple(t) for t in wordList)
    print('\n Total Word Count : %d') %(len(Corpus))
    

In [9]:
def get_similarity(Corpus,ds,keyword='cognition'):
    """
    Returns the most similar words, determined by neighborhood in corpus
    
    Attributes
    ----------
    
    Corpus : concatenated list of abstracts
    ds : ouptut of lisc
    keyword : query
    
    NOTE: There is a bug in text.similarity in NLTK2.0, we want to use this function to return similar 
    words in a ranked fashion 
    """
    #similar_words = Corpus.similar(keyword)
    #return similar_words
    #similar_words = Corpus._word_contex_index.similar_words(keyword)
    #if similar_words != None:
     #   print '\n'.join(word for word in similar_words)
    #else: 
     #   print 'No matches.'
        
    idx = nltk.text.ContextIndex([word.lower() for word in Corpus])
    simWords = []
    for word in nltk.word_tokenize(keyword):
        simWords.append(idx.similar_words(word))
        if word != None:
            print '\n'.join(idx.similar_words(word))
    return list(itertools.chain.from_iterable(simWords))

In [16]:
# just a toy example for now 
CA_words = ['abstract reasoning','attention','arousal']
def CA_word_presence(Corpus,CA_terms):
    """
    Returns the words from CA that are present in the CA
    
    TO DO: Generalize to entering tasks 
    
    Attributes
    ----------
    
    Corpus : concatenated list of abstracts
    """    
    mask = []
    for ca_idx in xrange(len(CA_terms)):
        if CA_terms[ca_idx] in Corpus:
            mask.append(True)
        else:
            mask.append(False)
            
    PresentTerms = list(compress(CA_words, mask))
    print '\n'.join(word for word in PresentTerms)
    
    return PresentTerms
    

### Scratchpad

In [ ]:
"""
TO DO:

1. Test functions with lisc instead of erpsc
2. Find out the CA terms can be checked within the corpus
3. Submit a pull request to add a version of this notebook to NHW repo
"""

In [10]:
Corpus = get_corpus(dat)

Iterating over 4992 articles for extracting words


In [11]:
get_common_words(Corpus,10)

Term       Frequency  
----       ---------  
p300       9910       
patients   3650       
amplitude  3050       
stimuli    2503       
subjects   2444       
latency    2436       
study      2431       
potentials 2408       
auditory   2286       
task       2268       

 Total Word Count : 442877


In [19]:
sim = get_similarity(Corpus,dat)

cognitive
working
plus
integration
structure
effects
potentials
age
stimuli
abilities
followed
findings
p3b
executive
brain
signs
morphology
curcumin
children
prodromal


In [ ]:
type(Corpus)

In [20]:
def test_neighbors(Corpus,keyword)
    
    plain_list = list(Corpus)
    max_instances = 10

    indices = [i for i, x in enumerate(plain_list) if x == "p300"]

    check_first = indices[:max_instances]

    for i in range(max_instances):
        print('Occurence %d of p300 preceded by %s and followed by %s.') \
        %(check_first[i],plain_list[i-1],plain_list[i+1])

SyntaxError: invalid syntax (<ipython-input-20-e5f0dd8527dc>, line 1)

### Visualization with Gensim

In [32]:
import gensim
from gensim import corpora
import pandas as pd
from gensim.models import LdaModel
import pyLDAvis.gensim

In [13]:
textList = []
    
numArticles = dat.n_articles
for AIdx in xrange(numArticles):
    if dat.words[AIdx] != None:
        textList.append(dat.words[AIdx])
        

In [14]:
dictionary = corpora.Dictionary(textList)
dictionary.save('dictionary.dict')

In [15]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in textList]
corpora.MmCorpus.serialize('corpus.mm', doc_term_matrix)

In [16]:
Lda = gensim.models.ldamodel.LdaModel

ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

ldamodel.save('topic.model')

In [17]:
loading = LdaModel.load('topic.model')
print(loading.print_topics(num_topics=2, num_words=4))

[(0, u'0.012*"p300" + 0.008*"bci" + 0.005*"eeg" + 0.005*"system"'), (1, u'0.030*"p300" + 0.022*"patients" + 0.012*"latency" + 0.010*"cognitive"')]


In [27]:
new['topic']

0    0.012*"p300" + 0.008*"bci" + 0.005*"eeg" + 0.0...
1    0.030*"p300" + 0.022*"patients" + 0.012*"laten...
2    0.020*"p300" + 0.012*"stimuli" + 0.010*"task" ...
Name: topic, dtype: object

In [28]:
pyLDAvis.enable_notebook()
import gensim

In [29]:
d = gensim.corpora.Dictionary.load('dictionary.dict')
c = gensim.corpora.MmCorpus('corpus.mm')
lda = gensim.models.LdaModel.load('topic.model')

In [33]:
data = pyLDAvis.gensim.prepare(lda, c, d)

In [34]:
data

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      46.091484        1       1 -0.076289  0.121217
1      35.025752        1       2 -0.120224 -0.104403
0      18.882763        1       3  0.196513 -0.016814, topic_info=      Category         Freq            Term        Total  loglift  logprob
term                                                                      
17956  Default  3449.000000        patients  3449.000000  30.0000  30.0000
9994   Default  2560.000000         stimuli  2560.000000  29.0000  29.0000
16252  Default  1750.000000        stimulus  1750.000000  28.0000  28.0000
13180  Default  1136.000000        controls  1136.000000  27.0000  27.0000
18572  Default   683.000000             bci   683.000000  26.0000  26.0000
455    Default  2156.000000            task  2156.000000  25.0000  25.0000
5006   Default  2804.000000         latency  2804.000000  24.0000  24.0000
12777  Default  1949.000000       cognitive  1949.000000  23.0000  23.0000
12896  Default   797.000000             age   797.000000  22.0000  22.0000
7480   Default   688.000000   schizophrenia   688.000000  21.0000  21.0000
8660   Default  1394.000000           group  1394.000000  20.0000  20.0000
5188   Default  3137.000000       amplitude  3137.000000  19.0000  19.0000
287    Default  2226.000000        auditory  2226.000000  18.0000  18.0000
4976   Default  1458.000000          target  1458.000000  17.0000  17.0000
7777   Default  1727.000000      processing  1727.000000  16.0000  16.0000
9050   Default   365.000000           cells   365.000000  15.0000  15.0000
10106  Default  1043.000000          normal  1043.000000  14.0000  14.0000
16599  Default   697.000000               p   697.000000  13.0000  13.0000
3831   Default   721.000000          system   721.000000  12.0000  12.0000
11198  Default   935.000000       latencies   935.000000  11.0000  11.0000
1783   Default   301.000000      expression   301.000000  10.0000  10.0000
8047   Default   280.000000            cell   280.000000   9.0000   9.0000
4449   Default   698.000000       condition   698.000000   8.0000   8.0000
14518  Default  9907.000000            p300  9907.000000   7.0000   7.0000
938    Default   423.000000        disorder   423.000000   6.0000   6.0000
3909   Default   423.000000        accuracy   423.000000   5.0000   5.0000
1373   Default   500.000000           years   500.000000   4.0000   4.0000
2933   Default   256.000000       interface   256.000000   3.0000   3.0000
17264  Default  1229.000000     significant  1229.000000   2.0000   2.0000
2350   Default   454.000000          method   454.000000   1.0000   1.0000
...        ...          ...             ...          ...      ...      ...
1406    Topic3    77.403139       chromatin    78.009403   1.6591  -6.9851
8645    Topic3    76.886992            pcaf    77.493590   1.6591  -6.9918
4033    Topic3    82.266006         nuclear    82.916028   1.6591  -6.9242
16576   Topic3    97.876732        nicotine    99.013565   1.6554  -6.7504
1783    Topic3   290.737568      expression   301.566192   1.6304  -5.6617
142     Topic3   224.779533  classification   260.054282   1.5212  -5.9190
14478   Topic3   185.826692        approach   222.556930   1.4866  -6.1093
2542    Topic3   183.466560           paper   226.048010   1.4582  -6.1221
2350    Topic3   303.455218          method   454.032989   1.2640  -5.6189
3831    Topic3   411.526202          system   721.744362   1.1051  -5.3143
3909    Topic3   274.739323        accuracy   423.516888   1.2342  -5.7183
8548    Topic3   296.117966           based   483.741335   1.1761  -5.6434
3167    Topic3   209.600531         methods   298.460159   1.3135  -5.9889
14647   Topic3   168.510232        proposed   228.840489   1.3609  -6.2071
7741    Topic3   416.074242             eeg  1176.198804   0.6277  -5.3033
17580   Topic3   244.483539             use   495.572030   0.9604  -5.8350
16877   Topi